![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, you will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx` (Excel files).

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed


In [1]:
# We've loaded the necessary packages for you in the first cell. Please feel free to add as many cells as you like!
suppressMessages(library(dplyr)) # This line is required to check your answer correctly
options(readr.show_types = FALSE) # This line is required to check your answer correctly
library(readr)
library(readxl)
library(stringr)

# Load CSV file for prices
airbnb_price <- read_csv('data/airbnb_price.csv', show_col_types=FALSE)

# Load Excel file for room types
airbnb_room_type <- read_excel('data/airbnb_room_type.xlsx')

# Load TSV file for review dates
airbnb_last_review <- read_tsv('data/airbnb_last_review.tsv', show_col_types=FALSE)

# Merge the three data frames together into one
listings <- airbnb_price %>%
  inner_join(airbnb_room_type, by = "listing_id") %>%
  inner_join(airbnb_last_review, by = "listing_id")

# What are the dates of the earliest and most recent reviews? 
# In order to use a function like max()/min() on last_review date column, format needs to be converted
review_dates <- listings %>%
  # Convert to date using the format 'Month DD YYYY'
  mutate(last_review_date = as.Date(last_review, format = "%B %d %Y")) %>%
  # Use max() and min() to take the latest and earliest dates
  summarize(first_reviewed = min(last_review_date),
            last_reviewed = max(last_review_date))

# How many of the listings are private rooms? 
# Since there are differences in capitalization, make capitalization consistent
private_room_count <- listings %>%
  mutate(room_type = str_to_lower(room_type)) %>%
  # Then count the number of each room_type
  count(room_type) %>%
  # Get row containing count for private rooms only
  filter(room_type == "private room") 

# Extract number of rooms
nb_private_rooms <- private_room_count$n


# What is the average listing price? 
# To convert price to numeric, remove "dollars" from each value
avg_price <- listings %>%
  mutate(price_clean = str_remove(price, " dollars") %>%
        as.numeric()) %>%
  # Take the mean of price_clean
  summarize(avg_price = mean(price_clean)) %>%
  # Convert from a tibble to a single number
  as.numeric()

# Load solution values into solution tibble:
# Note first_reviewed and last_reviewed columns in 
# review_dates were created earlier
review_dates$nb_private_rooms = nb_private_rooms
review_dates$avg_price = round(avg_price, 2)

print(review_dates)

# A tibble: 1 × 4
  first_reviewed last_reviewed nb_private_rooms avg_price
  <date>         <date>                   <int>     <dbl>
1 2019-01-01     2019-07-09               11356      142.
